<a href="https://colab.research.google.com/github/ctclumak/posts/blob/master/Etsy_Scraping_with_BS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests, re
import time,random
from bs4 import BeautifulSoup
import pandas as pd
import json
from json import loads

In [ ]:
query = "wedding+accessories"
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'} 
base_url = "https://www.etsy.com/search/?q=%s&ref=pagination&page=%d"
urls=[]

for i in range(1,5):
    url = base_url % (query,i)  
    urls.append(url)
  
for url in urls:
    soup_search = requests.get(url,headers=headers)
    soup=BeautifulSoup(soup_search.content,"html5lib")
    #This is the listing id list
    listing_id = soup.find_all("a")

    #This holds the listing url
    list_id_records = []
    keywords_records = []      
  
  #this gather listing url by listing id and adding to website address
    for listing in listing_id:
        list_id = (listing.get("data-listing-id"))
        if list_id != None:
            url_product = "http://www.etsy.com/listing/" + str(list_id) +"/"
            list_id_records.append(url_product)

    for list_id in list_id_records:
        etsy_page_product = requests.get(list_id)
        soup_product = BeautifulSoup(etsy_page_product.content, "html.parser")
        list_id_no = soup_product.select("a.listing-link")[0].get("data-listing-id")
        title = soup_product.find("div", {"data-component":"listing-page-title-component"}).text.strip()
        #seller = soup_product.select(".wt-mb-xs-1 p a.wt-text-link-no-underline")[0].find("span").text.strip().replace("\n"," ")
        price = soup_product.find("p", {"class":"wt-text-title-03 wt-mr-xs-2"}).text.strip().replace("\n"," ").replace("Price: ","").replace("+","")
        try:
          total_sales = soup_product.select(".wt-display-inline-flex-xs a.wt-text-link-no-underline")[0].find("span").text.strip().replace("\n"," ").replace("sales", "").strip()
        except:
          total_sales =0
        description = soup_product.find("p", {"class":"wt-text-body-01 wt-break-word"}).text.strip().replace("\n"," ")
        product_url = "http://www.etsy.com/listing/" + str(list_id_no) +"/"
        try: 
          review = soup_product.find("h3", {"class":"wt-text-body-03"}).text.strip().replace("\n",",").replace(",","").replace("shop","")
        except:
          review =0
        #website = soup_product.select(".wt-mb-xs-1 p a.wt-text-link-no-underline")[0].get("href")
        keywords_records.append((list_id_no, title, price, total_sales,description, product_url, review))
        df = pd.DataFrame(keywords_records, columns =["List_ID", "Title","Current_Price", "Product_Sales", "Product_Detail", "Product_list", "Total_Review"])
df_main = pd.DataFrame()
df_main = pd.concat([df_main,df])
time.sleep(random.randint(1,3))
df_main.to_csv('etsy_products.csv', index=False, encoding='utf-8')
        
len(keywords_records)